# Modeling of relative Yield, P-Uptake and P-Balance

Marc PErez  
February 13, 2025

In [ ]:
#| code-fold: false
RES <- readRDS("data/RES.rds")
Dmlr <- RES$nlme.coef.mrg
# d <- RES$data

## Setup

In [ ]:
#| code-fold: true
#| code-summary: "Benchmark helper function"
library(mlr3verse, quietly = TRUE)

mse <- msrs(c("regr.mse"))

if (!interactive())
  lgr::get_logger("mlr3")$set_threshold("warn")

get_benchi_table <- function(tasks, nfolds = 5) {
  set.seed(123)
  learners <- lrns(c("regr.featureless", "regr.lm", "regr.xgboost", "regr.ranger"))
  learners$regr.xgboost$param_set$set_values(
    eta = 0.04, 
    nrounds = 300, 
    max_depth = 2
  )

  benchi <- xfun::cache_rds({
    benchmark(benchmark_grid(
      tasks, 
      learners, 
      rsmp("cv", folds = nfolds)
    ))
  }, 
  file = "benchmark.rds", 
  dir = "cache/",
  hash = list(tasks, nfolds)
  )
  
  res <- tidyr::pivot_wider(benchi$aggregate(mse), 
    id_cols = task_id,
    names_from = learner_id,
    values_from = regr.mse
  ) |> as.data.frame()
  
  rownames(res) <- res$task_id
  res <- res[, -1]
  colnames(res) <- gsub("regr.", "", colnames(res))
  stopifnot(any(colnames(res) == "featureless"))
  res <- 1 - res / res$featureless
  res[, -1, drop = FALSE] |> round(3)
}

Testing prediction quality using

-   Linear models
-   Random forests (default parameters)
-   XGBoost (with parameter tuning)

**Weather Variables:**

In [ ]:
#| echo: false
Weather_vars <- c(
  "anavg_temp", "ansum_prec",
  "juvdev_prec", "juvdev_sun",
  "ansum_sun", "juvdev_temp"
)
stopifnot(all(Weather_vars %in% names(Dmlr)))
Weather_vars

[1] "anavg_temp"  "ansum_prec"  "juvdev_prec" "juvdev_sun"  "ansum_sun"  
[6] "juvdev_temp"

**Phosphor Variable sets:**

In [ ]:
#| echo: false
P_var_sets <- list(
  onlyweather = NULL,
  k = "k",
  PS = "PS_log",
  kPS = c("PS_log", "k", "kPS_log"),
  AAE10 = "soil_0_20_P_AAE10_log",
  CO2 = "soil_0_20_P_CO2_log",
  AAE10_CO2 = c("soil_0_20_P_AAE10_log", "soil_0_20_P_CO2_log"),
  AAE10_CO2_kPS = c("soil_0_20_P_AAE10_log", "soil_0_20_P_CO2_log", "PS_log", "k", "kPS_log"),
  CO2_kPS = c("soil_0_20_P_CO2_log", "PS_log", "k", "kPS_log")
)

**Response Variables**

In [ ]:
Y_vars <- c("Ymain_rel", "annual_P_uptake", "annual_P_balance")

## With Weather data

### TODO: Group - cross validation

    \(nam){
      mytsk <- as_task_regr(
        Dmlr[complete.cases(Dmlr$Ymain_rel), c(y, Weather_vars, P_var_sets[[nam]], "Site")],
        target = y,
        id = nam)
      mytsk$set_col_roles("Site", "group")
      mytsk
    }

Algorithm learns to predict location from weather since we do not do stratified cross-validation (leaving out locations).

### Ymain_rel

In [ ]:
#| code-fold: true
y <- "Ymain_rel"
lapply(names(P_var_sets), \(nam) as_task_regr(
    Dmlr[complete.cases(Dmlr$Ymain_rel),c(y, Weather_vars, P_var_sets[[nam]])],
    target = y,
    id = nam)) |>
  get_benchi_table() |> knitr::kable()

                       lm   xgboost   ranger
  --------------- ------- --------- --------
  onlyweather       0.201     0.518    0.520
  k                 0.186     0.476    0.487
  PS                0.234     0.603    0.585
  kPS               0.265     0.569    0.555
  AAE10             0.294     0.530    0.582
  CO2               0.259     0.569    0.578
  AAE10_CO2         0.288     0.515    0.594
  AAE10_CO2_kPS     0.347     0.596    0.607
  CO2_kPS           0.275     0.530    0.575


|               |    lm | xgboost | ranger |
|:--------------|------:|--------:|-------:|
| onlyweather   | 0.201 |   0.518 |  0.520 |
| k             | 0.186 |   0.476 |  0.487 |
| PS            | 0.234 |   0.603 |  0.585 |
| kPS           | 0.265 |   0.569 |  0.555 |
| AAE10         | 0.294 |   0.530 |  0.582 |
| CO2           | 0.259 |   0.569 |  0.578 |
| AAE10_CO2     | 0.288 |   0.515 |  0.594 |
| AAE10_CO2_kPS | 0.347 |   0.596 |  0.607 |
| CO2_kPS       | 0.275 |   0.530 |  0.575 |

### annual_P_uptake

In [ ]:
#| code-fold: true
y <- "annual_P_uptake"
lapply(names(P_var_sets), \(nam) as_task_regr(
    Dmlr[complete.cases(Dmlr$annual_P_uptake),c(y, Weather_vars, P_var_sets[[nam]])],
    target = y,
    id = nam)) |>
  get_benchi_table()  |> knitr::kable()

                       lm   xgboost   ranger
  --------------- ------- --------- --------
  onlyweather       0.468     0.649    0.648
  k                 0.456     0.564    0.609
  PS                0.477     0.594    0.617
  kPS               0.459     0.560    0.474
  AAE10             0.503     0.619    0.621
  CO2               0.481     0.615    0.642
  AAE10_CO2         0.484     0.566    0.580
  AAE10_CO2_kPS     0.465     0.528    0.423
  CO2_kPS           0.487     0.536    0.428


|               |    lm | xgboost | ranger |
|:--------------|------:|--------:|-------:|
| onlyweather   | 0.468 |   0.649 |  0.648 |
| k             | 0.456 |   0.564 |  0.609 |
| PS            | 0.477 |   0.594 |  0.617 |
| kPS           | 0.459 |   0.560 |  0.474 |
| AAE10         | 0.503 |   0.619 |  0.621 |
| CO2           | 0.481 |   0.615 |  0.642 |
| AAE10_CO2     | 0.484 |   0.566 |  0.580 |
| AAE10_CO2_kPS | 0.465 |   0.528 |  0.423 |
| CO2_kPS       | 0.487 |   0.536 |  0.428 |

### annual_P_balance

In [ ]:
#| code-fold: true
y <- "annual_P_balance"
lapply(names(P_var_sets), \(nam) as_task_regr(
    Dmlr[complete.cases(Dmlr$annual_P_balance),c(y, Weather_vars, P_var_sets[[nam]])],
    target = y,
    id = nam)) |>
  get_benchi_table()  |> knitr::kable()

                        lm   xgboost   ranger
  --------------- -------- --------- --------
  onlyweather        0.011     0.120    0.121
  k                 -0.008     0.313    0.127
  PS                 0.310     0.647    0.608
  kPS                0.282     0.609    0.617
  AAE10              0.225     0.510    0.524
  CO2                0.225     0.532    0.591
  AAE10_CO2          0.225     0.546    0.585
  AAE10_CO2_kPS      0.335     0.617    0.596
  CO2_kPS            0.302     0.626    0.631


|               |     lm | xgboost | ranger |
|:--------------|-------:|--------:|-------:|
| onlyweather   |  0.011 |   0.120 |  0.121 |
| k             | -0.008 |   0.313 |  0.127 |
| PS            |  0.310 |   0.647 |  0.608 |
| kPS           |  0.282 |   0.609 |  0.617 |
| AAE10         |  0.225 |   0.510 |  0.524 |
| CO2           |  0.225 |   0.532 |  0.591 |
| AAE10_CO2     |  0.225 |   0.546 |  0.585 |
| AAE10_CO2_kPS |  0.335 |   0.617 |  0.596 |
| CO2_kPS       |  0.302 |   0.626 |  0.631 |

## Without Weather data

In [ ]:
if("onlyweather" %in% names(P_var_sets))
  P_var_sets <- P_var_sets[-1]

xgboost & ranger are no good in this setting since only very few variables available

### Ymain_rel

In [ ]:
#| code-fold: true
y <- "Ymain_rel"
lapply(names(P_var_sets), \(nam) as_task_regr(
    Dmlr[complete.cases(Dmlr$Ymain_rel),c(y, P_var_sets[[nam]])],
    target = y,
    id = nam)) |>
  get_benchi_table()  |> knitr::kable()

                        lm   xgboost   ranger
  --------------- -------- --------- --------
  k                 -0.005    -0.200   -0.312
  PS                 0.060    -0.141   -0.221
  kPS                0.054    -0.343   -0.353
  AAE10              0.099     0.104   -0.009
  CO2                0.068    -0.086   -0.226
  AAE10_CO2          0.073     0.067    0.064
  AAE10_CO2_kPS      0.072    -0.045   -0.059
  CO2_kPS            0.023    -0.168   -0.196


|               |     lm | xgboost | ranger |
|:--------------|-------:|--------:|-------:|
| k             | -0.005 |  -0.200 | -0.312 |
| PS            |  0.060 |  -0.141 | -0.221 |
| kPS           |  0.054 |  -0.343 | -0.353 |
| AAE10         |  0.099 |   0.104 | -0.009 |
| CO2           |  0.068 |  -0.086 | -0.226 |
| AAE10_CO2     |  0.073 |   0.067 |  0.064 |
| AAE10_CO2_kPS |  0.072 |  -0.045 | -0.059 |
| CO2_kPS       |  0.023 |  -0.168 | -0.196 |

### annual_P_uptake

In [ ]:
#| code-fold: true
y <- "annual_P_uptake"
lapply(names(P_var_sets), \(nam) as_task_regr(
    Dmlr[complete.cases(Dmlr$annual_P_uptake),c(y, P_var_sets[[nam]])],
    target = y,
    id = nam)) |>
  get_benchi_table()  |> knitr::kable()

                        lm   xgboost   ranger
  --------------- -------- --------- --------
  k                 -0.006    -0.265   -0.434
  PS                -0.006    -0.278   -0.485
  kPS               -0.006    -0.225   -0.317
  AAE10              0.021    -0.139   -0.398
  CO2                0.005    -0.126   -0.276
  AAE10_CO2          0.017    -0.194   -0.310
  AAE10_CO2_kPS      0.020    -0.229   -0.363
  CO2_kPS           -0.054    -0.206   -0.295


|               |     lm | xgboost | ranger |
|:--------------|-------:|--------:|-------:|
| k             | -0.006 |  -0.265 | -0.434 |
| PS            | -0.006 |  -0.278 | -0.485 |
| kPS           | -0.006 |  -0.225 | -0.317 |
| AAE10         |  0.021 |  -0.139 | -0.398 |
| CO2           |  0.005 |  -0.126 | -0.276 |
| AAE10_CO2     |  0.017 |  -0.194 | -0.310 |
| AAE10_CO2_kPS |  0.020 |  -0.229 | -0.363 |
| CO2_kPS       | -0.054 |  -0.206 | -0.295 |

### annual_P_balance

In [ ]:
#| code-fold: true
y <- "annual_P_balance"
lapply(names(P_var_sets), \(nam) as_task_regr(
    Dmlr[complete.cases(Dmlr$annual_P_balance),c(y, P_var_sets[[nam]])],
    target = y,
    id = nam)) |>
  get_benchi_table()  |> knitr::kable()

                       lm   xgboost   ranger
  --------------- ------- --------- --------
  k                 0.011     0.175    0.160
  PS                0.309     0.274    0.178
  kPS               0.305     0.279    0.244
  AAE10             0.147     0.062   -0.102
  CO2               0.189     0.170    0.085
  AAE10_CO2         0.192     0.062    0.015
  AAE10_CO2_kPS     0.313     0.235    0.201
  CO2_kPS           0.313     0.274    0.243


|               |    lm | xgboost | ranger |
|:--------------|------:|--------:|-------:|
| k             | 0.011 |   0.175 |  0.160 |
| PS            | 0.309 |   0.274 |  0.178 |
| kPS           | 0.305 |   0.279 |  0.244 |
| AAE10         | 0.147 |   0.062 | -0.102 |
| CO2           | 0.189 |   0.170 |  0.085 |
| AAE10_CO2     | 0.192 |   0.062 |  0.015 |
| AAE10_CO2_kPS | 0.313 |   0.235 |  0.201 |
| CO2_kPS       | 0.313 |   0.274 |  0.243 |

In [ ]:
cor(Dmlr$annual_P_balance, Dmlr$PS) # 0.54389

[1] 0.5438943

[1] 0.4823593

[1] 0.07067821

We did manage to have high predictive power for weather. This could also be due to our regression models recovering location&year from it and hence still overfitting on the test set.

Without Weather data we only managed for annual balance to get some predictive power (30%). Since we the balance is uptake - fert_P, this means that we mostly predicted fert_P. Interestingly PS is best to predict this quantity

### Legacy Code

In [ ]:
#| code-summary: "XGBoost - Parameter Tuning"
#| code-fold: true
#| eval: false

# Get parameter estimates for XGBoost
t <- as_task_regr(
  subset(Dmlr[complete.cases(Dmlr$annual_P_balance),], 
    select = c("annual_P_balance", P_var_sets$AAE10_CO2_kPS#, Weather_vars
    )),
  target = "annual_P_balance"
)

l <- lrn("regr.xgboost",
  nrounds = 500  # More iterations due to lower learning rate
)

# Create search space
ps <- ps(
  max_depth = p_int(2, 4),
  eta = p_dbl(0.001, 0.3, tags = "logscale")
)

# Setup tuning
instance <- ti(
  task = t,
  learner = l,
  resampling = rsmp("cv", folds = 3),
  measure = msr("regr.mse"),
  terminator = trm("none"),
  search_space = ps
)

# Grid search
tuner <- mlr3tuning::tnr("grid_search")
tuner$optimize(instance)
instance$result

Ymain_rel max_depth eta learner_param_vals x_domain regr.mse <int> <num> <list> <list> <num> 1: 2 0.067444 \<list\[5\]\> \<list\[2\]\> 177.18

P uptake max_depth eta learner_param_vals x_domain regr.mse <int> <num> <list> <list> <num> 1: 2 0.034222 \<list\[5\]\> \<list\[2\]\> 137.41

annual_P_balance max_depth eta learner_param_vals x_domain regr.mse <int> <num> <list> <list> <num> 1: 2 0.034222 \<list\[5\]\> \<list\[2\]\> 145.21

In [ ]:
# nlme.coef$kPS <- nlme.coef$k * nlme.coef$PS
# 
# 
# nlme.coef.mrg <- merge(nlme.coef,allP[allP$year>=2017,],by = "uid")
# # add log-transformed versions
# Dmlr$kPS_log <- log(Dmlr$kPS)
# Dmlr$PS_log <- log(Dmlr$PS)
# Dmlr$soil_0_20_P_AAE10_log <- log(Dmlr$soil_0_20_P_AAE10)
# Dmlr$soil_0_20_P_CO2_log <- log(Dmlr$soil_0_20_P_CO2)
# 
# Dmlr$k



subset(Dmlr, select = c("Ymain_rel", P_var_sets$AAE10_CO2_kPS, Weather_vars))

    Ymain_rel soil_0_20_P_AAE10_log soil_0_20_P_CO2_log     PS_log          k
1       84.81              2.332144        -1.272965676 -2.9145162 0.20192074
2       92.91              2.388763        -1.237874356 -2.9145162 0.20192074
3          NA              2.332144        -1.272965676 -2.9145162 0.20192074
4          NA              2.406945        -1.171182982 -2.9145162 0.20192074
5          NA              2.197225        -1.078809661 -2.9145162 0.20192074
6       72.33              2.116256        -1.237874356 -2.9145162 0.20192074
7      106.32              2.484907        -1.171182982 -2.6264304 0.23314970
8       84.97              2.341806        -1.108662625 -2.6264304 0.23314970
9          NA              2.292535        -0.941608540 -2.6264304 0.23314970
10      82.00              2.322388        -1.021651248 -2.6264304 0.23314970
11         NA              2.341806        -1.108662625 -2.6264304 0.23314970
12         NA              2.379546        -1.108662625 -2.62643

# Methods

we used machine learning methods to assess how much information different sets of variables (c.f. `P_var_sets`) have each on the dependent variable (Puptake, Y-rel, P-balance), how redundant this information is. The machine learning methods to quantify the predictive power of different variable sets are: i) ordinary least squares (OLS) as a baseline; ii) XGBoost (gradient boosting with tree-based models and hyperparameter tuning for learning rate and tree depth) (arxiv:1603.02754); iii) Random Forests (with default parameters) (doi:10.1023/A:1010933404324). Computations were performed using the mlr3 framework (doi:10.21105/joss.01903). Performance was measured as percentage of explained variance on hold-out data via 5-fold cross-validation, calculated as (1 - MSE/Variance(y)), where MSE represents mean squared error.

We tried adjusting for weather variables but it seems that the ML-methods rather reconstruct the site-specific patterns….